# Background on Taproot

## Where does "taproot" come from?

Taproot is a series of bitcoin BIPS (bitcoin improvement proposals) that were merged into bitcoin-core in 2021.

It was most recent bitcoin soft-fork and it went live in November 2021.

The BIPS can be found on github. They are as follows:

- BIP340, Schnorr Signatures https://github.com/bitcoin/bips/blob/master/bip-0340.mediawiki
- BIP341, Taproot Segwit v1 https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki
- BIP342, Validating Taproot Spends https://github.com/bitcoin/bips/blob/master/bip-0342.mediawiki
- BIP350, Bech32m https://github.com/bitcoin/bips/blob/master/bip-0350.mediawiki

## What did "taproot" change?

Taproot added and modified many things in bitcoin. Including

- A new signature algorithm (Schnorr)
- A new version of Script opcodes (Segwit v1)
- A new bitcoin address checksum (bech32m)
- Removed some opcodes we know and love (OP_CHECKMULTISIG)
- Added a new opcode (OP_CHECKSIGADD)
- Added a new scripthash algorithm

# Finding Taproot in the Wild

Let's go to mempool.space and find some taproot addresses in the wild!



## How to spot a taproot address

The bech32 address has a `p` immediately following the `1`

Find a taproot address on mempool dot space.

What's the "Type" for this address?


Copy paste the "ScriptPubKey (HEX)" for the address you found into the pyblock below.

In [3]:
scriptpubkey = "51203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953"

In [4]:
print(scriptpubkey)

51203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953


In [5]:
len(scriptpubkey) // 2

34

This is a series of opcodes. If you pass this to `bitcoin-cli decodescript` what does it print back as the asm?

In [6]:
!bitcoin-cli -regtest decodescript 51203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953

{
  "asm": "1 3905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953",
  "desc": "addr(bcrt1p8yzasx690dpw340trtjjsryrca8mm5s29eqpcqrhhnj5z7gfe9fsysf0jl)#95u636fj",
  "address": "bcrt1p8yzasx690dpw340trtjjsryrca8mm5s29eqpcqrhhnj5z7gfe9fsysf0jl",
  "type": "witness_v1_taproot"
}


In [44]:
output = {
  "asm": "1 3905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953",
  "desc": "addr(bcrt1p8yzasx690dpw340trtjjsryrca8mm5s29eqpcqrhhnj5z7gfe9fsysf0jl)#95u636fj",
  "address": "bcrt1p8yzasx690dpw340trtjjsryrca8mm5s29eqpcqrhhnj5z7gfe9fsysf0jl",
  "type": "witness_v1_taproot"
}

In [45]:
print(output['asm'])

1 3905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953


In [46]:
output['asm'].split(" ")

['1', '3905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953']

In [47]:
len(output['asm'].split(" ")[1]) // 2

32

In [48]:
print(output['type'])

witness_v1_taproot


In [49]:
x_bytes = bytes.fromhex(output['asm'].split(' ')[1])
print('hex:\t\t', x_bytes.hex())

x_val = int.from_bytes(x_bytes, 'big')
print('int val:\t', x_val)

hex:		 3905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953
int val:	 25792158117016797743484614046835601668965634494428223733945610235101691103571


## Finding the secp256k1 pubkey from a P2TR address

Now that we have the x-only pubkey, let's figure out what point that is on the secp256k1 curve.

We can do that using `lift_x_pubkey`. This takes an integer and returns a point.

In my case, I'll use `coincurve`, a python library for expressing + doing math with public keys.

In [50]:
p = 115792089237316195423570985008687907853269984665640564039457584007908834671663
def lift_x(x_value):
    assert x_value < p
    # find y squared (y2 = x^3 + 7)
    y2 = (x_value ** 3 + 7) % p
    # find the square root of y
    y = pow(y2, (p+1)//4, p)
    assert (y ** 2 % p) == y2
    # if y is odd, find even y
    if y % 2 != 0:
        y = p - y
    return coincurve.PublicKey.from_point(x_value, y)

In [51]:
import coincurve

point = lift_x(x_val)
print(point.format().hex())
print(point.point())

023905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953
(25792158117016797743484614046835601668965634494428223733945610235101691103571, 4191453718700008539754125939901750860298491226938149986069530358203844164198)


## Let's Get Familiar With The Tools!

In [52]:
from codes import parse_tx_bytes

tx = "020000000001013c29211e2e6b9a594f28fd9e8b650dd2f64e7024d398fbc4c98f3b8800cef1350000000000ffffffff0358020000000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c95358020000000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953eba60d00000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c9530140bc93e2b4ae198c6326737b3e1525715109b6fa65cac52be9c1889408ba2d3ddeac9af8c944513905275280c9e8ac5fb6c83a7100137173c3e6a402857cb4c6de00000000"

In [53]:
print(parse_tx_bytes(tx))

AssertionError: 

In [54]:
from pprint import pprint

In [55]:
pprint(parse_tx_bytes(tx))

AssertionError: 

First things first, let's update the `parse_tx_bytes` method to be able to handle segwit (any version) transactions.

In [56]:
def parse_witness_bytes(data):
    count, size = parse_compact_size(data)
    ptr = size
    witnesses = []
    for _ in range(0, count):
        witlen, size = parse_compact_size(data[ptr:])
        ptr += size
        witnesses.append(data[ptr:ptr+witlen])
        ptr += witlen

    return witnesses, ptr

In [57]:
from codes import parse_compact_size, parse_input_bytes, parse_output_bytes

def parse_tx_bytes_mine(tx_hex):
  tx_bytes = bytes.fromhex(tx_hex)
  tx = {}
  ptr = 0
  tx['version'] = tx_bytes[0:4]
  ptr += 4

  if tx_bytes[ptr] == 0x00:
    assert tx_bytes[ptr+1] == 0x01
    tx['marker_flag'] = bytes([0x00, 0x01])
    ptr += 2

  count, size = parse_compact_size(tx_bytes[ptr:])
  ptr += size 
  tx['inputs'] = []
  for _ in range(0, count):
    inputx, size = parse_input_bytes(tx_bytes[ptr:])
    ptr += size
    tx['inputs'].append(inputx)
 
  count, size = parse_compact_size(tx_bytes[ptr:])
  ptr += size
  tx['outputs'] = []
  for _ in range(0, count):
    outputx, size = parse_output_bytes(tx_bytes[ptr:])
    ptr += size
    tx['outputs'].append(outputx)

  if 'marker_flag' in tx:
    tx['witnesses'] = []
    for _ in range(0, len(tx['inputs'])):
        witness, size = parse_witness_bytes(tx_bytes[ptr:])
        ptr += size
        tx['witnesses'].append(witness)

  tx['locktime'] = tx_bytes[ptr:]
  return tx

In [58]:
tx_bytes = bytes.fromhex(tx)
ptr = 4
print(tx_bytes.hex())
print(tx_bytes[ptr:].hex())

020000000001013c29211e2e6b9a594f28fd9e8b650dd2f64e7024d398fbc4c98f3b8800cef1350000000000ffffffff0358020000000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c95358020000000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953eba60d00000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c9530140bc93e2b4ae198c6326737b3e1525715109b6fa65cac52be9c1889408ba2d3ddeac9af8c944513905275280c9e8ac5fb6c83a7100137173c3e6a402857cb4c6de00000000
0001013c29211e2e6b9a594f28fd9e8b650dd2f64e7024d398fbc4c98f3b8800cef1350000000000ffffffff0358020000000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c95358020000000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953eba60d00000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c9530140bc93e2b4ae198c6326737b3e1525715109b6fa65cac52be9c1889408ba2d3ddeac9af8c944513905275280c9e8ac5fb6c83a7100137173c3e6a402857cb4c6de00000000


In [59]:
pprint(parse_tx_bytes_mine(tx))

{'inputs': [{'scriptSig': b'',
             'sequence': b'\xff\xff\xff\xff',
             'txid': b'<)!\x1e.k\x9aYO(\xfd\x9e\x8be\r\xd2\xf6Np$'
                     b'\xd3\x98\xfb\xc4\xc9\x8f;\x88\x00\xce\xf15',
             'vout': b'\x00\x00\x00\x00'}],
 'locktime': b'\x00\x00\x00\x00',
 'marker_flag': b'\x00\x01',
 'outputs': [{'amount': b'X\x02\x00\x00\x00\x00\x00\x00',
              'scriptPubKey': b'Q 9\x05\xd8\x1bE{B\xe8\xd5\xeb\x1a\xe5(\x0c'
                              b'\x83\xc7O\xbd\xd2\n.@\x1c\x00w\xbc\xe5Ay\t'
                              b'\xc9S'},
             {'amount': b'X\x02\x00\x00\x00\x00\x00\x00',
              'scriptPubKey': b'Q 9\x05\xd8\x1bE{B\xe8\xd5\xeb\x1a\xe5(\x0c'
                              b'\x83\xc7O\xbd\xd2\n.@\x1c\x00w\xbc\xe5Ay\t'
                              b'\xc9S'},
             {'amount': b'\xeb\xa6\r\x00\x00\x00\x00\x00',
              'scriptPubKey': b'Q 9\x05\xd8\x1bE{B\xe8\xd5\xeb\x1a\xe5(\x0c'
                              b'\x83\xc7O

In [60]:
txdata = parse_tx_bytes_mine(tx)

In [61]:
assert len(txdata['witnesses'][0]) == 1

In [62]:
print(txdata['witnesses'][0][0].hex())

bc93e2b4ae198c6326737b3e1525715109b6fa65cac52be9c1889408ba2d3ddeac9af8c944513905275280c9e8ac5fb6c83a7100137173c3e6a402857cb4c6de


In [63]:
sig = txdata['witnesses'][0][0]

In [64]:
print(sig.hex())

bc93e2b4ae198c6326737b3e1525715109b6fa65cac52be9c1889408ba2d3ddeac9af8c944513905275280c9e8ac5fb6c83a7100137173c3e6a402857cb4c6de


In [65]:
print(len(sig))

64


## Validating a Signature

What message did this signature sign?

Finding the "sighash" of that message. 

Take the bitcoin transaction that the signature was embedded into. From this transaction, we'll create a hash.

That hash of the bitcoin transaction "message digest" which is the hash that the signature "signed".

In [66]:
print(tx)

020000000001013c29211e2e6b9a594f28fd9e8b650dd2f64e7024d398fbc4c98f3b8800cef1350000000000ffffffff0358020000000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c95358020000000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953eba60d00000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c9530140bc93e2b4ae198c6326737b3e1525715109b6fa65cac52be9c1889408ba2d3ddeac9af8c944513905275280c9e8ac5fb6c83a7100137173c3e6a402857cb4c6de00000000


In [67]:
pprint(txdata)

{'inputs': [{'scriptSig': b'',
             'sequence': b'\xff\xff\xff\xff',
             'txid': b'<)!\x1e.k\x9aYO(\xfd\x9e\x8be\r\xd2\xf6Np$'
                     b'\xd3\x98\xfb\xc4\xc9\x8f;\x88\x00\xce\xf15',
             'vout': b'\x00\x00\x00\x00'}],
 'locktime': b'\x00\x00\x00\x00',
 'marker_flag': b'\x00\x01',
 'outputs': [{'amount': b'X\x02\x00\x00\x00\x00\x00\x00',
              'scriptPubKey': b'Q 9\x05\xd8\x1bE{B\xe8\xd5\xeb\x1a\xe5(\x0c'
                              b'\x83\xc7O\xbd\xd2\n.@\x1c\x00w\xbc\xe5Ay\t'
                              b'\xc9S'},
             {'amount': b'X\x02\x00\x00\x00\x00\x00\x00',
              'scriptPubKey': b'Q 9\x05\xd8\x1bE{B\xe8\xd5\xeb\x1a\xe5(\x0c'
                              b'\x83\xc7O\xbd\xd2\n.@\x1c\x00w\xbc\xe5Ay\t'
                              b'\xc9S'},
             {'amount': b'\xeb\xa6\r\x00\x00\x00\x00\x00',
              'scriptPubKey': b'Q 9\x05\xd8\x1bE{B\xe8\xd5\xeb\x1a\xe5(\x0c'
                              b'\x83\xc7O

## Find the SigMsg for a transaction

Here we use the BIP341 Signature Validation Rules (https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki#user-content-Common_signature_message) Common signature message definition to calculate the SIGHASH_DEFAULT for a given transaction.

We'll use this to compute the sighash for this input, which we need to validate the signature.

In [68]:
from hashlib import sha256
from codes import size_compact_size

def sigmsg_default(tx_hex, input_index, annex_bytes, amounts_bytes, scriptpubkeys_bytes, ext_flag):
    txdata = parse_tx_bytes_mine(tx_hex)
    
    result = b''
    result += bytes([0x00]) # sighash_flag
    result += txdata['version']
    result += txdata['locktime']

    all_input_outpoints = b''
    for inp in txdata['inputs']:
        # txid || vout
        all_input_outpoints += inp['txid'] + inp['vout']    
    sha_prevouts = sha256(all_input_outpoints).digest()
    result += sha_prevouts
    
    sha_amounts = sha256(b''.join(amounts_bytes)).digest()        
    result += sha_amounts
    
    spks = b''
    for spk in scriptpubkeys_bytes:
        spks += size_compact_size(len(spk)) + spk
    sha_scriptpubkeys = sha256(spks).digest()
    result += sha_scriptpubkeys
    
    all_sequences = [ i['sequence'] for i in txdata['inputs'] ]
    sha_sequences = sha256(b''.join(all_sequences)).digest()
    result += sha_sequences
    
    all_outputs = b''
    for o in txdata['outputs']:
        # amount || compact_size(scriptpubkey) || scriptpubkey
        all_outputs += o['amount']
        all_outputs += size_compact_size(len(o['scriptPubKey']))
        all_outputs += o['scriptPubKey']
    sha_outputs = sha256(all_outputs).digest()
    result += sha_outputs
    
    # data about this input
    annex_present = 1 if annex_bytes else 0
    spend_type = 2 * ext_flag + annex_present
    result += (spend_type).to_bytes(1, 'little')
    result += (input_index).to_bytes(4, 'little')
    
    if (annex_bytes):
        size_bytes = size_compact_size(len(annex_bytes))
        sha_annex = sha256(size_bytes + annex_bytes).digest()
        result += sha_annex
    
    # data about this output
    # noop for SIGHASH_DEFAULT
    
    assert len(result) <= 206
    return result

In [69]:
spent_from_tx = parse_tx_bytes_mine("02000000000101d79ca7455775b3fc7bf65eae536faa22e6248b863247cee30765fabb4fafadb80100000000ffffffff02dcab0e00000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953944be70400000000225120dc2e293b7717ece7898fefc8fd7f6da13e8fb56efac0bc3bd75183304da6ad1701409dab8ddd9c703d1acb6736bc5260b23e64999cb130b9b3ca6c2e69aa084073302b66f0ecb5b4a191618951c022da2c429ac3638613c43bbb88a761634c2cb85300000000")

print(spent_from_tx['outputs'][0])

{'amount': b'\xdc\xab\x0e\x00\x00\x00\x00\x00', 'scriptPubKey': b'Q 9\x05\xd8\x1bE{B\xe8\xd5\xeb\x1a\xe5(\x0c\x83\xc7O\xbd\xd2\n.@\x1c\x00w\xbc\xe5Ay\t\xc9S'}


In [70]:
amounts_bytes = [spent_from_tx['outputs'][0]['amount']]
print("amounts:", [x.hex() for x in amounts_bytes])

scriptpubkeys_bytes = [spent_from_tx['outputs'][0]['scriptPubKey']]
print("scriptpubkeys:", [x.hex() for x in scriptpubkeys_bytes], "\n")

ext_flag = 0
annex_bytes = None
input_index = 0

sigmsg = sigmsg_default(tx, input_index, annex_bytes, amounts_bytes, scriptpubkeys_bytes, ext_flag)

print("sigmsg:", sigmsg.hex())

amounts: ['dcab0e0000000000']
scriptpubkeys: ['51203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953'] 

sigmsg: 000200000000000000002b9b64a25d11192d251c2e088e7cbc65889963ff67244102b5fc7cb1b4feb51c4eb25d1adc8a44ed6dd54bca7d3d790a7b9e66ef9a8416836882124ad71bb507d41f17121f975dfae4cb8a052b3e34576e1c272bf7e17dfe484be8d1f96317ad95131bc0b799c0b1af477fb14fcf26a6a9f76079e48bf090acb7e8367bfd0e5cdaded8d0fdd6bcb42703ac71156c5d658a47acaa0b90924087e48059d5f1c30000000000


## Computing the Sighash

Now that we have the sighash message, we can calculate the actual "sighash" for the signature.

The `sighash` for Taproot key path spending signature valdiation is as follows:

```
sighash = tag_hash(b'TapSighash', bytes([0x00]) + sigmsg)
```

We'll need to implement the `tag_hash` function, then pass our `sigmsg` into this to find the `sighash`.

In [71]:
from hashlib import sha256

def tag_hash(tag, data_bytes):
    taghash = sha256(tag).digest()
    return sha256(taghash + taghash + data_bytes).digest()

In [72]:
sighash = tag_hash(b'TapSighash', bytes([0x00]) + sigmsg)
print(sighash.hex())

c09a12ac690db9c044c0df972aec9f574cf12677b80b1b82e910bf9e16f3b115


## Verifying the Sighash

Now that we've got the `sighash` we're finally ready to verify the signature.

For this, we'll need the `verify` function, which is defined in BIP340.

In [73]:
# there are three inputs to the verify function: pubkey, message, signature
pk = x_val
m = sighash
sig = sig

print(x_val)

25792158117016797743484614046835601668965634494428223733945610235101691103571


In [74]:
from codes import n

def verify(x_val, m, sig):
    P = lift_x(x_val)
    assert len(sig) == 64
    r = int.from_bytes(sig[:32], 'big')
    if r >= p:
        return False
    s = int.from_bytes(sig[32:], 'big')
    if s >= n:
        return False
    
    P_bytes = (x_val).to_bytes(32, 'big')
    challenge_hash = tag_hash(b'BIP0340/challenge', sig[:32] + P_bytes + m)
    e = int.from_bytes(challenge_hash, 'big') % n
    
    # e * P
    eP = P.multiply(e.to_bytes(32, 'big'))
    # s * G
    sG = coincurve.PrivateKey.from_int(s).public_key
    neg_eP = invert_point(eP)
    
    # R = sG - eP = sG + -eP
    R = coincurve.PublicKey.combine_keys([sG, neg_eP])
    
    if not has_even_y(R):
        return False
    
    return R.point()[0] == r

In [75]:
def invert_point(pt):
    x, y = pt.point()
    yneg = p - y
    return coincurve.PublicKey.from_point(x, yneg)

In [76]:
def has_even_y(pt):
    return pt.point()[1] % 2 == 0

In [77]:
sig = txdata['witnesses'][0][0]
verify(x_val, m, sig)

True

## Signing a message

Now that we've been able to successfully verify a message, let's finish implementing the second method for Schnorr: the Sign method.

This takes a private key and a message and returns a valid signature.

```
sig = Sign(privkey, msg)
```

In [78]:
def scalar_to_pubkey(scalar_val):
    assert scalar_val != 0 and scalar_val < n
    P = coincurve.PrivateKey.from_int(scalar_val).public_key
    result = scalar_val if has_even_y(P) else n - scalar_val
    
    return result, P


def sign_msg(privkey_int, msg, nonce_int):
    d, P = scalar_to_pubkey(privkey_int)
    k, R = scalar_to_pubkey(nonce_int)
    
    bytes_R = (R.point()[0].to_bytes(32, 'big'))
    bytes_P = (P.point()[0].to_bytes(32, 'big'))
    challenge_hash = tag_hash(b'BIP0340/challenge', bytes_R + bytes_P + msg)
    e = int.from_bytes(challenge_hash, 'big') % n
    
    # ~~~BEGIN SCHNORR PART~~~
    s = (k + e * d) % n
    # ~~~ END SCHNORR PART ~~~
    
    sig = bytes_R + s.to_bytes(32, 'big')
    assert verify(P.point()[0], m, sig)
    return sig

In [79]:
privkey = 10
nonce = 5
sig = sign_msg(privkey, m, nonce)

print(sig.hex())

2f8bde4d1a07209355b4a7250a5c5128e88b84bddc619ab7cba8d569b240efe4de48d2d7d8f794b7e45e4701c5385b94d87543365be474d9941f15d4ecf211a2


## Taproot: Exploring the Depths

We've just done a Taproot keypath spend verification (~P2PK)

Now we're going to look at how to build out scripts for taproot!

We're going to make our first taproot script. (~P2WSH)

What are the steps we need to do for this:

- figure out what scripts can spend our bitcoin
- make a tree out of them (just like christmas!)
- build a merkle root
- pick a internal pubkey
- make a tweak from the merkle root
- add the tweak to the internal pubkey to get our external pubkey
- lock money up to external pubkey

### Step 1: Figure out what scripts can spend our bitcoin

- Everyone in class should make a script!
- Each individually lock up coins to the script (single script spend)
- Send all the scripts to Lisa, and then we'll build a tree out of them

#### Writing a taproot script

nifty's script: OP_4 OP_ADD OP_8 OP_EQUAL  (requires OP_4 to unlock)

Convert to script: 
        OP_4 OP_ADD OP_8 OP_EQUAL
        54935887
        
        

In [80]:
nifty = '54935887'
casey =  'AA201ccf6c5e6212f524600fb6b20275cd6ae26dc6b812cd1f84dd3ff7d86b1937d187'
jose = '54935887'
dpp = '930200088763a820150faa5b485225f681b179f710cb169b92b401f954392eb30e677624135233f08768'
mk = '76769393010987'
damian = 'a820528f99cd13f2d438556c6f6a803458a963519d2419a40dd685597c0919d81f588754935587'
chris = '53935987'
nate = 'a82041ef4bb0b23661e66301aac36066912dac037827b4ae63a7b1165a5aa93ed4eb87'

In [81]:
scripts = [('nifty', nifty), ('casey', casey), ('jose', jose), ('dpp', dpp), ('mk', mk), ('damian', damian), ('chris', chris), ('nate', nate)]

### Step 2: Make a tree out of your scripts

For this first round, we're just going to use a single script!

Every tree has leaves! Every script in our tree is a leaf.

Our tree is just [nifty]

In [82]:
tree = [nifty]

In [83]:
## Step 3: Build a Merkle root

In [84]:
### Step 3a: Write a function to compute a leaf (hash)

In [85]:
def make_leaf(script_bytes):
    leaf_version = 0xc0
    # leaf version + len script + script
    data = bytes([leaf_version]) + size_compact_size(len(script_bytes)) + script_bytes
    return tag_hash(b'TapLeaf', data)

In [86]:
tree_bytes = [ bytes.fromhex(x) for x in tree ]

In [87]:
leaf = make_leaf(tree_bytes[0])
print(leaf.hex())

7ce84a5b9c296529029e07e014ffdfb9bb11d201bd84e4528d4ae46d6160ead6


In [88]:
for name, script in [ (a,bytes.fromhex(x)) for a,x in scripts]:
    int_leaf = make_leaf(script)
    print(name, int_leaf.hex())

nifty 7ce84a5b9c296529029e07e014ffdfb9bb11d201bd84e4528d4ae46d6160ead6
casey bcf0ffea8d704f70610ddffb14a2cd74cbde7b4bf116605143f43fcdac862858
jose 7ce84a5b9c296529029e07e014ffdfb9bb11d201bd84e4528d4ae46d6160ead6
dpp 7199c3d01803160ae132c34a5aaa9e5ba44725b0e6a483caa4de6804e77e2b01
mk fa2b5d06e6fdf4ce947e00511fa6dd92294996846ec8a3b0b80d9363f0dec93a
damian 5f305f38bd7306856e35ac66b1f0c528a76af3ecfb777c4afef45bb80b2a0a6a
chris 8ddbf7add8c35b385346ea0f1c3fa93bbfd9583c75ee097fe0c03c64cceccd0a
nate 002a01ee216a40f3d68710b87d7fb0654a7d540e9baa3f5ba5a06170b82ed08d


In this first example, where we only have one script, the "merkle root" is the leaf of the single script.

Our root = leaf.

In [89]:
root = leaf
print(leaf.hex())

7ce84a5b9c296529029e07e014ffdfb9bb11d201bd84e4528d4ae46d6160ead6


### Step 4: Pick an internal pubkey

Considerations to make when picking an *internal* pubkey:

- Does it need to be spendable?
- If yes, who has the keys to it?
- If no, how can I prove it's unspendable?


In our case, our key needs to be unspendable. We need to pick a point that we don't know the private key to.

The spec (BIP341) recommends using the following algorithm (or something like it) to pick a point where the private key is unknowns.

> One example of such a point is H = lift_x(0x50929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0) which is constructed by taking the hash of the standard uncompressed encoding of the secp256k1 base point G as X coordinate. In order to avoid leaking the information that key path spending is not possible it is recommended to pick a fresh integer r in the range 0...n-1 uniformly at random and use H + rG as internal key. It is possible to prove that this internal key does not have a known discrete logarithm with respect to G by revealing r to a verifier who can then reconstruct how the internal key was created.

In [90]:
# We define the nums_point as
# H(G) => sha256(uncompressed)
def nums_point(r=0):
    Gx, Gy = coincurve.PrivateKey.from_int(1).public_key.point()
    data = bytes([0x04]) + Gx.to_bytes(32, 'big') + Gy.to_bytes(32, 'big')
    h = sha256(data).digest()
    h_int = int.from_bytes(h, 'big')
    H = lift_x(h_int)
    if r > 0:
        assert r < n
        rG = coincurve.PrivateKey.from_int(r).public_key
        return coincurve.PublicKey.combine_keys([rG, H])
    return H

In [91]:
internal_pubkey = nums_point()
print(internal_pubkey.format().hex())

pubkey_bytes = internal_pubkey.point()[0].to_bytes(32, 'big')

0250929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0


### Step 5: Make a tweak from the Merkle Root

What is a tweak? Ideally, a tweak is really just a number (scalar).

We're gonna take the root and a pubkey, make a hash out of both of them and then convert this hash to a number.

We'll then use that number (scalar) as if it were a private key and find the curve point at that value.

In [92]:
def make_tweak_pubkey(pubkey, merkle_root_bytes):
    # Find the bytes for the x-value of the internal pubkey
    pubkey_bytes = pubkey.point()[0].to_bytes(32, 'big')
    
    # Compute the tweak hash (TapTweak tag + pubkey + merkle root)
    tweak_bytes = tag_hash(b'TapTweak', pubkey_bytes + merkle_root_bytes)
    # Convert that hash into a scalar (so we can make it a point)
    tweak_int = int.from_bytes(tweak_bytes, 'big')
    # Make sure our tweak is inside the "point range (n)"
    assert tweak_int < n
    
    # Multiply the tweak by the Generator Point (G) which gives us T
    T = coincurve.PrivateKey.from_int(tweak_int).public_key
    
    return T

In [93]:
T = make_tweak_pubkey(internal_pubkey, root)
print(T.format().hex())

03276d6adba4bf408222b0923821f9d6286d7d2e0750f99fab541e199c224e2b74


In [94]:
for name, script in [ (a,bytes.fromhex(x)) for a,x in scripts]:
    root = make_leaf(script)
    T = make_tweak_pubkey(internal_pubkey, root)
    print(name, T.format().hex())

nifty 03276d6adba4bf408222b0923821f9d6286d7d2e0750f99fab541e199c224e2b74
casey 02d50363f23b5712ddaf1e2f1ddbb754f95607e99e26a8983da8c44c7525c43434
jose 03276d6adba4bf408222b0923821f9d6286d7d2e0750f99fab541e199c224e2b74
dpp 02e134c26d8e2e52f1339507d00c698a840bf8dd8e6480b9b6fdc2d803805e4b50
mk 03154f265ff85914f6d7e8506663d89cf12fa2ac167946cfe97f5eec3f4a934420
damian 031dd4af1b86eae0f26b5235f33e121b82af19d25827a4b79432a3c9d52424129d
chris 0214decaa772191bdd99d46bf9ba0578f9dd97a900baf49aa55300c87cbfdd9223
nate 021410525faa15e71bb2290bb425043f220639ae89815e768ff9fcef55f017386b


Now we have our Tweak as a point. All we need to do is combine it with the internal pubkey to get an external pubkey.

### Step 6: Add Internal Pubkey to Tweak Pubkey

For some reason the spec makes everything more fancy that it needs to be.

The biggest difference is that we "promote" the internal pubkey to a x-only pubkey (even) before we add it to the tweak.

In [95]:
def make_external_pubkey(pubkey, T):    
    P = lift_x(pubkey.point()[0])
    Q = coincurve.PublicKey.combine_keys([P, T])
    return Q

In [96]:
nifty

nifty_leaf = make_leaf(bytes.fromhex(nifty))
print(internal_pubkey.format().hex())
print('nifty root:', nifty_leaf.hex())
nifty_T = make_tweak_pubkey(internal_pubkey, nifty_leaf)
print('nifty tweak', nifty_T.format().hex())
nifty_Q = make_external_pubkey(internal_pubkey, nifty_T)
print('nifty!', nifty_Q.format().hex())

0250929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0
nifty root: 7ce84a5b9c296529029e07e014ffdfb9bb11d201bd84e4528d4ae46d6160ead6
nifty tweak 03276d6adba4bf408222b0923821f9d6286d7d2e0750f99fab541e199c224e2b74
nifty! 029970f40840ae66e7f420877f2b48d331eda1da215d05ec1de6bd6cd4ea45ab9f


In [97]:
for name, script in [ (a, bytes.fromhex(x)) for a,x in scripts]:
    root = make_leaf(script)
    T = make_tweak_pubkey(internal_pubkey, root)
    Q = make_external_pubkey(internal_pubkey, T)
    print(name, Q.format().hex())

nifty 029970f40840ae66e7f420877f2b48d331eda1da215d05ec1de6bd6cd4ea45ab9f
casey 025a62e8ead0b9f60f5a91fd69d4044b531e086823cbab7b67ccbb312509eb9fb5
jose 029970f40840ae66e7f420877f2b48d331eda1da215d05ec1de6bd6cd4ea45ab9f
dpp 028df34c51844750dc133f9521285ec78c92a8ba336d435951e8f1dc68611aca58
mk 03311eb2f7a8ca651980285fb6bbf2c3d548191cdb9e6ba902562fffe35fa28aab
damian 03f933d817c628bb0e249a44138753de965d271fb2b347daf54b24225cac8bb1a3
chris 030c2d7cf15ed86872eb9957326fe706ec54e2d100e03505538c54cae5919453ee
nate 02806750c9417bbb6b017bd96a47bcf1372120310d0cee67e5eeff5cd5bf28bd0e


### Step 7: Lock bitcoin up to your external pubkey! (but first make a P2TR script)

We'll take Q, drop the parity byte at the front and make this into a P2TR script.

In [98]:
def make_p2tr(Q):
    Qx_bytes = Q.point()[0].to_bytes(32, 'big')
    witness_version = 0x51
    return bytes([witness_version]) + size_compact_size(len(Qx_bytes)) + Qx_bytes


print(make_p2tr(nifty_Q).hex())
nifty_script = make_p2tr(nifty_Q).hex()

51209970f40840ae66e7f420877f2b48d331eda1da215d05ec1de6bd6cd4ea45ab9f


In [99]:
# FIXME: add `jq` to nix-shell
address_arr = !bitcoin-cli -regtest decodescript "$nifty_script" | jq -r .address
print(address_arr[0])
address = address_arr[0]

bcrt1pn9c0gzzq4enw0apqsaljkjxnx8k6rk3pt5z7c80xh4kdf6j94w0s4fwuky


In [100]:
!bitcoin-cli -regtest createwallet base58

error code: -4
error message:
Wallet file verification failed. Failed to create database path '/home/niftynei/.bitcoin/regtest/wallets/base58'. Database already exists.


In [101]:
!bitcoin-cli -regtest -generate 101

{
  "address": "bcrt1qu7q40w60nztdhypxv4gmaz7k3m2qqmc0x6pcdy",
  "blocks": [
    "1e3d9ecb1dae24506e626033231358f1f136d266f2a3f2872de0b0499db92f6f",
    "0e82b9f4075734ff4d57ef88edc18b6c0eecf5a530cf772f4fa787b759582801",
    "2932fb071d3f5a94eb58e5d64e8c26b77a862bd025a3fc071030d7d8b63cec4f",
    "375ac4b8d39570d19e25f6ae8c7015fce52f06da283997a14e23434e94ae7fb3",
    "0ba0651171751a4ba8e344fd2017b411b54919c3dfdbb7d0e3c1b759f408da30",
    "5349689c89fd7d5f39857f30033bd0a0449f9094c921bdef29d9712b96c8b733",
    "5c2a3cb682423be2ec5f044b6ad81acfa5ac2be6ebcce3a6405946312b65a511",
    "3e336e899e76b0286ef9949e8d2670551d4203a3ef05b6b598f5bf43223a7cab",
    "632d49b01d3afc3f694366c8487353b6cff1e4c068882d68d550a713951699b9",
    "247d1b5648be7efec984f2dcbdd82a0a2d7c5be84a611eddd5f2c311e4b0c71e",
    "28c1e8cb80c2b2e1a05697aa20f317406c972f70926fed15dffe2d9fa40a1e04",
    "6b8ea6f5c5feae2c142df2a11faa2a1d209924520ea288bf8e799b84ced96486",
    "3ca673f68a4135351ba10f2d90273b81a96d0085748b55f91764c0

In [ ]:
!bitcoin-cli -regtest sendtoaddress "$address" 1.0

In [ ]:
!bitcoin-cli -regtest getrawtransaction d57032ce2ac09efc013ed6786d508e8912e3486d4fddab5e4982fcdb161155a4 true

## Spending a Tapscript!

We should have a transaction output with bitcoin in it that's locked to your custom written script.

Now we want to create a transaction that spends that output.

My output is in transaction d57032ce2ac09efc013ed6786d508e8912e3486d4fddab5e4982fcdb161155a4 and the index is 1

In other words the outpoint for my bitcoins is `d57032ce2ac09efc013ed6786d508e8912e3486d4fddab5e4982fcdb161155a4:1`

Let's a build a transaction that spends this!

In [ ]:
txid = 'd57032ce2ac09efc013ed6786d508e8912e3486d4fddab5e4982fcdb161155a4'
reverse_txid = bytes.fromhex(txid)[::-1].hex()
print(reverse_txid)

amount = (1 * 10 ** 8 - 400).to_bytes(8, 'little')
print(amount.hex())

In [ ]:
!bitcoin-cli -regtest getnewaddress "" bech32m

In [ ]:
!bitcoin-cli -regtest getaddressinfo bcrt1pywat5mnqc4pyvajq3hdmujza9dwzl3lefq26w6sdhtxfgk4wyyqqsvx823 | jq -r .scriptPubKey

version: 02000000
marker+flag: 0001
inputs: 01
    txid: a4551116dbfc82495eabdd4f6d48e312898e506d78d63e01fc9ec02ace3270d5
    vout: 01000000
    scriptSig: 00
    sequence: ffffffff
outputs: 01
    amounts: 70dff50500000000
    scriptPubKey: 22 512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae2100
witnesses:
    ........ unsolved problem ........
locktime: 00000000

In [ ]:
tx = "0200000001a4551116dbfc82495eabdd4f6d48e312898e506d78d63e01fc9ec02ace3270d50100000000ffffffff0170dff5050000000022512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae210000000000"
!bitcoin-cli -regtest decoderawtransaction "$tx"

### What's in the witness stack for a leaf spend?

We need to put into the witness stack:
 - the unlocking script
 - original leaf script
 - "control block": this has all the data you need to verify the external public key
 
 #### Nifty's Unlock Script is...
 
 Well my original leaf script

In [ ]:
!bitcoin-cli -regtest decodescript "$nifty" | jq .asm

Q: How do I unlock this? Great question.

Short A: 04
Long A: The number "4". In script, we'd use the opcode 'OP_4'
but in witnesses we can't use opcodes, so we just do a "push of the byte 04".

In [ ]:
!bitcoin-cli -regtest decodescript 0104

- Unlocking script: 04
- Original leaf data??: c0 + len(nifty) + 54935887 (variable: nifty)
- Control block

#### How do we figure out a control block?


A control block has three parts.

- version byte
- the x-only pubkey of the internal key
- proof of inclusion

##### Version Byte

We're going to need Q to figure this out.

The base value for the version byte is `c0`.
With the following modification: if Q is odd, add 1 to the base value.

If Q is even (02): 0xc0

If Q is odd (03):  0xc1

In [102]:
def control_block_version_byte(Q):
    val = 0xc0
    if Q.point()[1] % 2 != 0:
        val += 1
    return bytes([val])

print(control_block_version_byte(nifty_Q).hex())

c0


##### The x-only pubkey of the internal key

This should just be the internal pubkey, 32 bytes.

In [103]:
print(internal_pubkey.format()[1:].hex())

50929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0


###### Proof of Inclusion

The proof of inclusion for a leaf-only spend is empty.

In [140]:
control_block = control_block_version_byte(nifty_Q) + internal_pubkey.format()[1:]

print('witness data!')
print('03')
print('01 04')
print('04 ' + nifty)
print('21 ' + control_block.hex())

witness data!
03
01 04
04 54935887
21 c050929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0


version: 02000000
marker+flag: 0001
inputs: 01
    txid: a4551116dbfc82495eabdd4f6d48e312898e506d78d63e01fc9ec02ace3270d5
    vout: 01000000
    scriptSig: 00
    sequence: ffffffff
outputs: 01
    amounts: 70dff50500000000
    scriptPubKey: 22 512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae2100
witnesses:
03
    01 04
    04 54935887
    21 c050929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0
locktime: 00000000

In [105]:
spend_tx = '02000000000101a4551116dbfc82495eabdd4f6d48e312898e506d78d63e01fc9ec02ace3270d50100000000ffffffff0170dff5050000000022512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae2100030104045493588721c050929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac000000000'
#!bitcoin-cli -regtest decoderawtransaction "$spend_tx"

In [139]:
spend_tx

'02000000000101a4551116dbfc82495eabdd4f6d48e312898e506d78d63e01fc9ec02ace3270d50100000000ffffffff0170dff5050000000022512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae2100030104045493588721c050929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac000000000'

In [107]:
!bitcoin-cli -regtest testmempoolaccept '["02000000000101a4551116dbfc82495eabdd4f6d48e312898e506d78d63e01fc9ec02ace3270d50100000000ffffffff0170dff5050000000022512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae2100030104045493588721c050929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac000000000"]'

[
  {
    "txid": "65c5ee8463feeb72112a458216e8600e5f386a685cba5b63dcaa4a18d6c51327",
    "wtxid": "7b50f74e105ac8a955af55172c263b0807b946a79a54120a1380b195d148b469",
    "allowed": true,
    "vsize": 105,
    "fees": {
      "base": 0.00000400
    }
  }
]


### Let's build a taproot with mulitple leaves

Let's make a taproot with a bunch of leaves!

We have a set of scripts that we want to allow any one of them to spend this bitcoin.

Build a tree of the scripts, and then compute the merkle root.

In [136]:
scripts

[('nifty', '54935887'),
 ('casey',
  'AA201ccf6c5e6212f524600fb6b20275cd6ae26dc6b812cd1f84dd3ff7d86b1937d187'),
 ('jose', '54935887'),
 ('dpp',
  '930200088763a820150faa5b485225f681b179f710cb169b92b401f954392eb30e677624135233f08768'),
 ('mk', '76769393010987'),
 ('damian',
  'a820528f99cd13f2d438556c6f6a803458a963519d2419a40dd685597c0919d81f588754935587'),
 ('chris', '53935987'),
 ('nate',
  'a82041ef4bb0b23661e66301aac36066912dac037827b4ae63a7b1165a5aa93ed4eb87')]

In [278]:
balanced_tree = [[[nifty, jose], [casey, dpp]], [[mk, damian], [chris, nate]]]
unbalanced_tree = [ nifty, [ chris, [ jose, [ casey, [ dpp, [ mk, [ damian , nate ]]]]]]]

In [279]:
balanced_tree

[[['54935887', '54935887'],
  ['AA201ccf6c5e6212f524600fb6b20275cd6ae26dc6b812cd1f84dd3ff7d86b1937d187',
   '930200088763a820150faa5b485225f681b179f710cb169b92b401f954392eb30e677624135233f08768']],
 [['76769393010987',
   'a820528f99cd13f2d438556c6f6a803458a963519d2419a40dd685597c0919d81f588754935587'],
  ['53935987',
   'a82041ef4bb0b23661e66301aac36066912dac037827b4ae63a7b1165a5aa93ed4eb87']]]

In [280]:
unbalanced_tree

['54935887',
 ['53935987',
  ['54935887',
   ['AA201ccf6c5e6212f524600fb6b20275cd6ae26dc6b812cd1f84dd3ff7d86b1937d187',
    ['930200088763a820150faa5b485225f681b179f710cb169b92b401f954392eb30e677624135233f08768',
     ['76769393010987',
      ['a820528f99cd13f2d438556c6f6a803458a963519d2419a40dd685597c0919d81f588754935587',
       'a82041ef4bb0b23661e66301aac36066912dac037827b4ae63a7b1165a5aa93ed4eb87']]]]]]]

### Now that our tree(s) are defined, we need to calculate the merkle root

We need a function that given our tree, will return a merkle root.

In [ ]:
def make_leaf(script_bytes):
    leaf_version = 0xc0
    # leaf version + len script + script
    data = bytes([leaf_version]) + size_compact_size(len(script_bytes)) + script_bytes
    return tag_hash(b'TapLeaf', data)

In [228]:
def taptree_builder(tree):
    if isinstance(tree, str):
        script_bytes = bytes.fromhex(tree)
        leaf_hash = make_leaf(script_bytes)
        print("leaf:", leaf_hash.hex())
        return leaf_hash
    
    # calculate the branch hash. 
    assert len(tree) == 2
    left = taptree_builder(tree[0])
    right = taptree_builder(tree[1])
    
    # we have to order the left + right "alphabetically"
    # thank you arik :)
    if left > right:
        right, left = left, right
    
    branch_hash = tag_hash(b'TapBranch', left + right)
    print("branch:", branch_hash.hex())
    return branch_hash
    

In [229]:
balanced_root = taptree_builder(balanced_tree)
balanced_root.hex()

leaf: 7ce84a5b9c296529029e07e014ffdfb9bb11d201bd84e4528d4ae46d6160ead6
leaf: 7ce84a5b9c296529029e07e014ffdfb9bb11d201bd84e4528d4ae46d6160ead6
branch: c6f6f180831c02d2c477a6baa1e34fb0b49897543f149ae83fbd3deb89c8ae80
leaf: bcf0ffea8d704f70610ddffb14a2cd74cbde7b4bf116605143f43fcdac862858
leaf: 7199c3d01803160ae132c34a5aaa9e5ba44725b0e6a483caa4de6804e77e2b01
branch: 9542ca2491c786b840b98222e18c58ef4eda480755b4ee3424fced25c58150a5
branch: 111fbd5ec2591327c312c7db37a0722205156310354d11783777f41ceff2fe96
leaf: fa2b5d06e6fdf4ce947e00511fa6dd92294996846ec8a3b0b80d9363f0dec93a
leaf: 5f305f38bd7306856e35ac66b1f0c528a76af3ecfb777c4afef45bb80b2a0a6a
branch: 2869d5d001b985f60582de5e96d27d49bf19e111de5908b70a75d4c1c4354700
leaf: 8ddbf7add8c35b385346ea0f1c3fa93bbfd9583c75ee097fe0c03c64cceccd0a
leaf: 002a01ee216a40f3d68710b87d7fb0654a7d540e9baa3f5ba5a06170b82ed08d
branch: b95df7c5373a203b605e4c4773579b51f853078ba1a5f9db35df6527566c0c6d
branch: 8903a87b4aa61a20cc352975eab9ed92f780a37313c823561af39470021

'bf9d5551696d290b9b8cb134fef97bcc3e8a4e7812c9719fc8c3140947cd12cf'

In [171]:
unbalanced_root = taptree_builder(unbalanced_tree)
unbalanced_tree

leaf: 002a01ee216a40f3d68710b87d7fb0654a7d540e9baa3f5ba5a06170b82ed08d
leaf: 5f305f38bd7306856e35ac66b1f0c528a76af3ecfb777c4afef45bb80b2a0a6a
branch: f1da90107484360f6a99c3dd6bf46e0a9981881588e4210f59dc45bb923c87f0
leaf: fa2b5d06e6fdf4ce947e00511fa6dd92294996846ec8a3b0b80d9363f0dec93a
branch: e54d0e57313111b88e0c4ce1a579b7fde0352e6aa8403d79c0ee9ca50c389bbb
leaf: 7199c3d01803160ae132c34a5aaa9e5ba44725b0e6a483caa4de6804e77e2b01
branch: 2a67515502ffea3c0703a0479be3949c9947800b21d39e685e7e98ebdbab657b
leaf: bcf0ffea8d704f70610ddffb14a2cd74cbde7b4bf116605143f43fcdac862858
branch: 84186ded4ee597659c81a86e361a566259430fc71415c4b17b362a95c5ab243a
leaf: 7ce84a5b9c296529029e07e014ffdfb9bb11d201bd84e4528d4ae46d6160ead6
branch: 2a2af5481b5651304abeb64328c6ee695a34a486ddfa36f39e56995c9da1747a
leaf: 8ddbf7add8c35b385346ea0f1c3fa93bbfd9583c75ee097fe0c03c64cceccd0a
branch: 13eed6c9107796176054fa81be4bd6f6ec878cb3a918de1bc911be4c8e9b0566
leaf: 7ce84a5b9c296529029e07e014ffdfb9bb11d201bd84e4528d4ae46d616

['54935887',
 ['53935987',
  ['54935887',
   ['AA201ccf6c5e6212f524600fb6b20275cd6ae26dc6b812cd1f84dd3ff7d86b1937d187',
    ['930200088763a820150faa5b485225f681b179f710cb169b92b401f954392eb30e677624135233f08768',
     ['76769393010987',
      ['a820528f99cd13f2d438556c6f6a803458a963519d2419a40dd685597c0919d81f588754935587',
       'a82041ef4bb0b23661e66301aac36066912dac037827b4ae63a7b1165a5aa93ed4eb87']]]]]]]

In [130]:
internal_pubkey.format().hex()

'0250929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0'

In [142]:
balanced_tweak_key = make_tweak_pubkey(internal_pubkey, balanced_root)
balanced_tweak_key.format().hex()

'022a78497e11a16bfd866a87b6d09ed953a07988e10626f9313e27bb5cf51d9161'

In [144]:
unbalanced_tweak_key = make_tweak_pubkey(internal_pubkey, unbalanced_root)
unbalanced_tweak_key.format().hex()

'026524dcd9efddc696215edd34f6e9d96428a67f04208a5cb7e5b33187b9d01897'

In [145]:
Q_balanced = make_external_pubkey(internal_pubkey, balanced_tweak_key)
Q_unbalanced = make_external_pubkey(internal_pubkey, unbalanced_tweak_key)

In [155]:
p2tr_balanced = make_p2tr(Q_balanced).hex()
p2tr_balanced

'5120d5fe9ae05d009609889713d4376e18194cebd61105525fc52d53ca49ac1ca198'

In [158]:
p2tr_unbalanced = make_p2tr(Q_unbalanced).hex()
p2tr_unbalanced

'512098a7c168dfcc8928936a39ce374969ee121ff210b2350ca704c7293d903c17cd'

In [157]:
!bitcoin-cli -regtest decodescript "$p2tr_balanced" | jq -r .address

bcrt1p6hlf4czaqztqnzyhz02rwmscr9xwh4s3q4f9l3fd209yntqu5xvqh797tq


In [159]:
!bitcoin-cli -regtest decodescript "$p2tr_unbalanced" | jq -r .address

bcrt1pnznuz6xlejyj3ym2888rwjtfacfplusskg6sefcycu5nmypuzlxspdv3ku


In [180]:
# Balanced Tree Address Lock Up
#!bitcoin-cli -regtest sendtoaddress bcrt1p6hlf4czaqztqnzyhz02rwmscr9xwh4s3q4f9l3fd209yntqu5xvqh797tq 1.1
"3f4e1ffcb93fae8f1101b321616b1d9ca18cb909d02668bbfd76c062075fa01c"

'3f4e1ffcb93fae8f1101b321616b1d9ca18cb909d02668bbfd76c062075fa01c'

In [174]:
# Unbalanced Tree Address Lock Up
!bitcoin-cli -regtest sendtoaddress bcrt1pnznuz6xlejyj3ym2888rwjtfacfplusskg6sefcycu5nmypuzlxspdv3ku 1.2

3e1075a2dbd8f13e67d9944b464420a67e0bcd1db476b8e50a9d50aae27c76f3


In [177]:
!bitcoin-cli -regtest getrawtransaction 821c72b0a18427be1d80a8980057291e47f502ee0687e66ca5861f499449d567 true

{
  "txid": "821c72b0a18427be1d80a8980057291e47f502ee0687e66ca5861f499449d567",
  "hash": "2cc2dc65bcd8c4a55f59e5c8c9f40431e5c95b885603ae1670cd5cf0b4268c8f",
  "version": 2,
  "size": 246,
  "vsize": 165,
  "weight": 657,
  "locktime": 305,
  "vin": [
    {
      "txid": "e46663eb0fb1cd06a02de1597fbbf7126917e7733273345dc3ad2cd15f402761",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "30440220518adabd1d1703c6551b091c3b68c6bdaf6eb837610867bfe5bd7d60aca8cf0e0220267413adaa2b939faab17d3478e41ebd74fc68c94a90be10eed12a9c590edc6301",
        "02dd2e5d35d614b83d9319ae83a8a64728be5c8ce9da32a6c159b8fd48f8acc0b1"
      ],
      "sequence": 4294967294
    }
  ],
  "vout": [
    {
      "value": 23.90000000,
      "n": 0,
      "scriptPubKey": {
        "asm": "1 42cdd0d2869e107b14841859bb6a3141cb3bed2c9f07c3d48b41585d48b044f6",
        "desc": "addr(bcrt1pgtxap55xncg8k9yyrpvmk633g89nhmfvnuru84ytg9v96j9sgnmqlcps5q)#q8n0a5g0",
     

In [185]:
balanced_outpoint = ("821c72b0a18427be1d80a8980057291e47f502ee0687e66ca5861f499449d567", 1)
unbalanced_outpoint = ("3e1075a2dbd8f13e67d9944b464420a67e0bcd1db476b8e50a9d50aae27c76f3", 0)

txid_balanced = bytes.fromhex(balanced_outpoint[0])[::-1].hex()
txid_unbal = bytes.fromhex(unbalanced_outpoint[0])[::-1].hex()

print(txid_balanced)
print(txid_unbal)

67d54994491f86a56ce68706ee02f5471e29570098a8801dbe2784a1b0721c82
f3767ce2aa509d0ae5b876b41dcd0b7ea62044464b94d9673ef1d8dba275103e


## Let's Spend Some Things!

In [182]:
# amount unbalanced
(12 * 10 ** 7 - 400).to_bytes(8, 'little').hex()

'700c270700000000'

In [183]:
# amount balanced
(11 * 10 ** 7 - 400).to_bytes(8, 'little').hex()

'f0758e0600000000'

### Balanced Spend Transaction

```
version: 02000000
marker+flag: 0001
inputs: 01
    txid: 67d54994491f86a56ce68706ee02f5471e29570098a8801dbe2784a1b0721c82
    vout: 01000000
    scriptSig: 00
    sequence: ffffffff
outputs: 01
    amounts: f0758e0600000000
    scriptPubKey: 22 512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae2100
witnesses:
    03
        <unlock script>
        <leaf script>
        <control block!>
locktime: 00000000
```

In [296]:
leaf_script = bytes.fromhex(damian)
leaf_script.hex()

'a820528f99cd13f2d438556c6f6a803458a963519d2419a40dd685597c0919d81f588754935587'

In [198]:
unlock_script = b'Taproot Rocks!'
unlock_script.hex()

'546170726f6f7420526f636b7321'

In [289]:
def build_proof_of_inclusion(tree, script):
    if isinstance(tree, str):
        script_bytes = bytes.fromhex(tree)
        leaf_hash = make_leaf(script_bytes)
        return leaf_hash, [], script == tree
    
    # calculate the branch hash. 
    assert len(tree) == 2
    left_hash, left_proof, left_target = build_proof_of_inclusion(tree[0], script)
    right_hash, right_proof, right_target = build_proof_of_inclusion(tree[1], script)
    
    proof = []
    if left_target:
        left_proof.append(right_hash)
        proof = left_proof
    elif right_target:
        right_proof.append(left_hash)
        proof = right_proof
        
    # we have to order the left + right "alphabetically"
    # thank you arik :)
    if left_hash > right_hash:
        right_hash, left_hash = left_hash, right_hash
    branch_hash = tag_hash(b'TapBranch', left_hash + right_hash)
    return branch_hash, proof, left_target or right_target


_, proof_of_inclusion, _ = build_proof_of_inclusion(balanced_tree, damian)

print('proof', [ x.hex() for x in proof_of_inclusion])

proof ['fa2b5d06e6fdf4ce947e00511fa6dd92294996846ec8a3b0b80d9363f0dec93a', 'b95df7c5373a203b605e4c4773579b51f853078ba1a5f9db35df6527566c0c6d', '111fbd5ec2591327c312c7db37a0722205156310354d11783777f41ceff2fe96']


In [292]:
print(b''.join(proof_of_inclusion).hex())

control_block = control_block_version_byte(Q_balanced) + internal_pubkey.format()[1:] + b''.join(proof_of_inclusion)

fa2b5d06e6fdf4ce947e00511fa6dd92294996846ec8a3b0b80d9363f0dec93ab95df7c5373a203b605e4c4773579b51f853078ba1a5f9db35df6527566c0c6d111fbd5ec2591327c312c7db37a0722205156310354d11783777f41ceff2fe96


In [293]:
control_block.hex()

'c050929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0fa2b5d06e6fdf4ce947e00511fa6dd92294996846ec8a3b0b80d9363f0dec93ab95df7c5373a203b605e4c4773579b51f853078ba1a5f9db35df6527566c0c6d111fbd5ec2591327c312c7db37a0722205156310354d11783777f41ceff2fe96'

In [302]:
wits = [size_compact_size(len(unlock_script)) + unlock_script, 
        size_compact_size(len(leaf_script)) + leaf_script,
        size_compact_size(len(control_block)) + control_block]

print(size_compact_size(len(wits)).hex())
for witness in wits:
    print(witness.hex())

03
0e546170726f6f7420526f636b7321
27a820528f99cd13f2d438556c6f6a803458a963519d2419a40dd685597c0919d81f588754935587
81c050929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0fa2b5d06e6fdf4ce947e00511fa6dd92294996846ec8a3b0b80d9363f0dec93ab95df7c5373a203b605e4c4773579b51f853078ba1a5f9db35df6527566c0c6d111fbd5ec2591327c312c7db37a0722205156310354d11783777f41ceff2fe96


### Balanced Spend Transaction

```
version: 02000000
marker+flag: 0001
inputs: 01
    txid: 67d54994491f86a56ce68706ee02f5471e29570098a8801dbe2784a1b0721c82
    vout: 01000000
    scriptSig: 00
    sequence: ffffffff
outputs: 01
    amounts: f0758e0600000000
    scriptPubKey: 22 512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae2100
witnesses:
    03
        0e546170726f6f7420526f636b7321
        27a820528f99cd13f2d438556c6f6a803458a963519d2419a40dd685597c0919d81f588754935587
        81c050929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0fa2b5d06e6fdf4ce947e00511fa6dd92294996846ec8a3b0b80d9363f0dec93ab95df7c5373a203b605e4c4773579b51f853078ba1a5f9db35df6527566c0c6d111fbd5ec2591327c312c7db37a0722205156310354d11783777f41ceff2fe96
locktime: 00000000
```

In [306]:
tx = '0200000000010167d54994491f86a56ce68706ee02f5471e29570098a8801dbe2784a1b0721c820100000000ffffffff01f0758e060000000022512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae2100030e546170726f6f7420526f636b732127a820528f99cd13f2d438556c6f6a803458a963519d2419a40dd685597c0919d81f58875493558781c050929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0fa2b5d06e6fdf4ce947e00511fa6dd92294996846ec8a3b0b80d9363f0dec93ab95df7c5373a203b605e4c4773579b51f853078ba1a5f9db35df6527566c0c6d111fbd5ec2591327c312c7db37a0722205156310354d11783777f41ceff2fe9600000000'

In [305]:
!bitcoin-cli -regtest decoderawtransaction "$tx"

{
  "txid": "429c57e63e6cb1fe85fa665c9210149cb5dd415950e892b9357b39cd0d6cdaf3",
  "hash": "8109fa9970e0422327c6513d2afee32aceab17800faf2ff9a6e5cf5dafe5372d",
  "version": 2,
  "size": 282,
  "vsize": 141,
  "weight": 564,
  "locktime": 0,
  "vin": [
    {
      "txid": "821c72b0a18427be1d80a8980057291e47f502ee0687e66ca5861f499449d567",
      "vout": 1,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "546170726f6f7420526f636b7321",
        "a820528f99cd13f2d438556c6f6a803458a963519d2419a40dd685597c0919d81f588754935587",
        "c050929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0fa2b5d06e6fdf4ce947e00511fa6dd92294996846ec8a3b0b80d9363f0dec93ab95df7c5373a203b605e4c4773579b51f853078ba1a5f9db35df6527566c0c6d111fbd5ec2591327c312c7db37a0722205156310354d11783777f41ceff2fe96"
      ],
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": 1.09999600,
      "n": 0,
      "scriptPubKey": {
        "asm": "1 23baba

In [307]:
!bitcoin-cli -regtest testmempoolaccept '["0200000000010167d54994491f86a56ce68706ee02f5471e29570098a8801dbe2784a1b0721c820100000000ffffffff01f0758e060000000022512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae2100030e546170726f6f7420526f636b732127a820528f99cd13f2d438556c6f6a803458a963519d2419a40dd685597c0919d81f58875493558781c050929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0fa2b5d06e6fdf4ce947e00511fa6dd92294996846ec8a3b0b80d9363f0dec93ab95df7c5373a203b605e4c4773579b51f853078ba1a5f9db35df6527566c0c6d111fbd5ec2591327c312c7db37a0722205156310354d11783777f41ceff2fe9600000000"]'

[
  {
    "txid": "429c57e63e6cb1fe85fa665c9210149cb5dd415950e892b9357b39cd0d6cdaf3",
    "wtxid": "8109fa9970e0422327c6513d2afee32aceab17800faf2ff9a6e5cf5dafe5372d",
    "allowed": true,
    "vsize": 141,
    "fees": {
      "base": 0.00000400
    }
  }
]


### Unbalanced Spend Transaction

```
version: 02000000
marker+flag: 0001
inputs: 01
    txid: f3767ce2aa509d0ae5b876b41dcd0b7ea62044464b94d9673ef1d8dba275103e
    vout: 00000000
    scriptSig: 00
    sequence: ffffffff
outputs: 01
    amounts: 700c270700000000
    scriptPubKey: 22 512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae2100
witnesses:
    ........ unsolved problem ........
locktime: 00000000
```